# CSS5228 Final Project

In this notebook, we will start training our model

- Random Forest Base - MAE: 331763.892261509

In [42]:
import pandas as pd
import numpy as np
from pathlib import Path

from efficient_apriori import apriori
import matplotlib.pyplot as plt

In [85]:
df_train = pd.read_csv("../../../clean_data/train_preproc.csv", index_col=0)
df_train

,property_type,built_year,num_beds,num_baths,size_sqft,total_num_units,lat,lng,subzone,planning_area,...,mean_property_sqft,mean_planning_sqft,planning_area_mean,total_rooms,size_per_room,mean_property_type,distance,num_shopping_malls,tenure_99-year leasehold,tenure_freehold
0,hdb,1988,3.0,2.0,1115,116,1.414399,103.837196,yishun south,yishun,...,1079.757868,1231.474343,1.143726e+06,5.0,223.00,6.620452e+05,0.573567,2.0,1,0
1,hdb,1992,4.0,2.0,1575,375,1.372597,103.875625,serangoon north,serangoon,...,1079.757868,2514.468039,3.670975e+06,6.0,262.50,6.620452e+05,1.728895,3.0,1,0
2,condo,2022,4.0,6.0,3070,56,1.298773,103.895798,mountbatten,marine parade,...,1154.798804,2011.903265,4.159877e+06,10.0,307.00,2.919816e+06,1.315256,5.0,0,1
3,condo,2023,3.0,2.0,958,638,1.312364,103.803271,farrer court,bukit timah,...,1154.798804,2468.346271,5.576084e+06,5.0,191.60,2.919816e+06,0.723885,4.0,0,1
4,condo,2026,2.0,1.0,732,351,1.273959,103.843635,anson,downtown core,...,1154.798804,1590.161473,4.853464e+06,3.0,244.00,2.919816e+06,0.370022,16.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20091,condo,2026,2.0,2.0,635,605,1.385938,103.834466,tagore,ang mo kio,...,1154.798804,1634.952055,2.355193e+06,4.0,158.75,2.919816e+06,0.150007,4.0,1,0
20092,condo,2026,2.0,2.0,883,137,1.315948,103.857589,lavender,kallang,...,1154.798804,1106.259542,2.256485e+06,4.0,220.75,2.919816e+06,0.442631,6.0,0,1
20093,condo,2023,4.0,4.0,1378,340,1.315961,103.836848,moulmein,novena,...,1154.798804,1662.211716,3.637860e+06,8.0,172.25,2.919816e+06,0.422131,5.0,0,1
20094,hdb,2017,3.0,2.0,1205,402,1.440753,103.806671,woodlands east,woodlands,...,1079.757868,1229.137405,8.223176e+05,5.0,241.00,6.620452e+05,0.632423,8.0,1,0


In [104]:
feature_list = ['size_per_room','num_baths','num_beds','size_sqft', 'num_shopping_malls']
def preprocessingTrainData(feature_list, input_df):
    df_train1 = input_df[feature_list]
    bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000,1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000,
     2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900 ]
    labels =[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
    21, 22, 23, 24, 25, 26, 27, 28, 29]
    df_train1['size_per_room_binned'] = pd.cut(df_train1['size_per_room'], bins,labels=labels)
    df_train1["num_baths_str"] = df_train1.num_baths.apply(lambda s: 'num_baths_'+ str(int(s)))
    df_train1["num_beds_str"]= df_train1.num_beds.apply(lambda s: 'num_beds_'+ str(int(s)))
    df_train1["size_sqft_str"]= df_train1.size_sqft.apply(lambda s: 'size_sqft_'+ str(int(s)))
    df_train1["num_shopping_malls_str"]= df_train1.num_shopping_malls.apply(lambda s: 'num_shopping_malls_'+ str(int(s)))
    df_train1['size_per_room_binned_str'] = df_train1.size_per_room_binned.apply(lambda s: 'size_per_room_binned_str_'+ str(int(s)))
    df_train2 = df_train1[['size_per_room_binned_str','num_baths_str','num_beds_str','size_sqft_str', 'num_shopping_malls_str']]
    df_train2
    return df_train2


In [105]:
feature_list = ['size_per_room','num_baths','num_beds','size_sqft', 'num_shopping_malls']
# df_train1.min()
# #52.7
# df_train1.max()
# #2825.4
# #size_per_room
df_train2 = preprocessingTrainData(feature_list, df_train)
df_train2

/Users/akash/opt/miniconda3/envs/cs5242/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/akash/opt/miniconda3/envs/cs5242/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/akash/opt/miniconda3/envs/cs5242/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,size_per_room_binned_str,num_baths_str,num_beds_str,size_sqft_str,num_shopping_malls_str
0,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1115,num_shopping_malls_2
1,size_per_room_binned_str_3,num_baths_2,num_beds_4,size_sqft_1575,num_shopping_malls_3
2,size_per_room_binned_str_4,num_baths_6,num_beds_4,size_sqft_3070,num_shopping_malls_5
3,size_per_room_binned_str_2,num_baths_2,num_beds_3,size_sqft_958,num_shopping_malls_4
4,size_per_room_binned_str_3,num_baths_1,num_beds_2,size_sqft_732,num_shopping_malls_16
...,...,...,...,...,...
20091,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_635,num_shopping_malls_4
20092,size_per_room_binned_str_3,num_baths_2,num_beds_2,size_sqft_883,num_shopping_malls_6
20093,size_per_room_binned_str_2,num_baths_4,num_beds_4,size_sqft_1378,num_shopping_malls_5
20094,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1205,num_shopping_malls_8


In [99]:
ap_input_trans = df_train2.values.tolist()
itemsets, rules = apriori(ap_input_trans, min_support=0.1, min_confidence=0.4, max_length=3)
#itemsets


In [98]:
rules_rhs = filter(lambda rule: len(rule.lhs) >= 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{num_baths_2, num_beds_3} -> {size_per_room_binned_str_3} (conf: 0.633, supp: 0.187, lift: 1.303, conv: 1.400)
{num_beds_3, size_per_room_binned_str_3} -> {num_baths_2} (conf: 0.812, supp: 0.187, lift: 1.616, conv: 2.643)
{num_baths_2, size_per_room_binned_str_3} -> {num_beds_3} (conf: 0.691, supp: 0.187, lift: 1.759, conv: 1.963)
{num_beds_2, size_per_room_binned_str_2} -> {num_baths_2} (conf: 0.899, supp: 0.104, lift: 1.791, conv: 4.950)
{num_baths_2, num_beds_2} -> {size_per_room_binned_str_2} (conf: 0.673, supp: 0.104, lift: 2.024, conv: 2.039)
{num_baths_2, size_per_room_binned_str_2} -> {num_beds_2} (conf: 0.497, supp: 0.104, lift: 2.277, conv: 1.553)


## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [102]:
# Pick a row id of choice
row_id = 10
#row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row_location = df_train.iloc[row_id][["lat", "lng"]]
row = df_train.iloc[row_id]
test_data = pd.DataFrame([row])
test_data
# Just for printing it nicely, we create a new dataframe from this single row
# pd.DataFrame([row])

,property_type,built_year,num_beds,num_baths,size_sqft,total_num_units,lat,lng,subzone,planning_area,...,mean_property_sqft,mean_planning_sqft,planning_area_mean,total_rooms,size_per_room,mean_property_type,distance,num_shopping_malls,tenure_99-year leasehold,tenure_freehold
10,condo,1985,2.0,2.0,1733,280,1.322153,103.945223,bedok south,bedok,...,1154.798804,2287.615299,3.128573e+06,4.0,433.25,2.919816e+06,0.570617,2.0,1,0


In [106]:
feature_list = ['size_per_room','num_baths','num_beds','size_sqft', 'num_shopping_malls']
df_test_processed = preprocessingTrainData(feature_list, test_data)
df_test_processed

/Users/akash/opt/miniconda3/envs/cs5242/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/akash/opt/miniconda3/envs/cs5242/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/akash/opt/miniconda3/envs/cs5242/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,size_per_room_binned_str,num_baths_str,num_beds_str,size_sqft_str,num_shopping_malls_str
10,size_per_room_binned_str_5,num_baths_2,num_beds_2,size_sqft_1733,num_shopping_malls_2


In [101]:
# for rule in rules:
#     left_hand_side = rule.lhs    # or antecedent
#     right_hand_side = rule.rhs   # or consequent
#     support = rule.support
#     confidence = rule.confidence
#     lift = rule.lift
#     conviction = rule.conviction
#     print('{} => {} -- lift: {}'.format(left_hand_side, right_hand_side, lift))

In [107]:
threshold = 0.2
def get_pref_feature_bases(user_preferences):
    recommendation_to_user = []
    for rule in rules:
        left_hand_side = rule.lhs    # or antecedent
        right_hand_side = rule.rhs   # or consequent
        support = rule.support
        confidence = rule.confidence
        lift = rule.lift
        conviction = rule.conviction
        #print('{} => {} -- lift: {}'.format(left_hand_side, right_hand_side, lift))
        for pref in user_preferences:
            # print(pref, rule.lhs[0])
            if pref in rule.lhs:
                # print('{} => {} -- lift: {}'.format(left_hand_side, right_hand_side, lift))
                if lift > threshold:
                  recommendation_to_user.append(rule)

    return recommendation_to_user
            

def get_top_recommendations(user_preferences):
    recommendation_to_user = get_pref_feature_bases(user_preferences)
    # if score > threshold:
    #     recommendation_to_user.append(recom)

    return recommendation_to_user


final_rules = get_top_recommendations(df_test_processed.iloc[0])
final_rules
    

[{num_beds_2} -> {num_baths_2},
 {num_baths_2} -> {num_beds_3},
 {num_shopping_malls_2} -> {num_baths_2},
 {num_baths_2} -> {size_per_room_binned_str_2},
 {num_baths_2} -> {size_per_room_binned_str_3},
 {num_beds_2} -> {size_per_room_binned_str_2},
 {num_beds_2, size_per_room_binned_str_2} -> {num_baths_2},
 {num_baths_2, size_per_room_binned_str_2} -> {num_beds_2},
 {num_baths_2, num_beds_2} -> {size_per_room_binned_str_2},
 {num_baths_2, num_beds_2} -> {size_per_room_binned_str_2},
 {num_beds_2} -> {num_baths_2, size_per_room_binned_str_2},
 {num_baths_2, size_per_room_binned_str_3} -> {num_beds_3},
 {num_baths_2, num_beds_3} -> {size_per_room_binned_str_3}]

In [108]:
rules_rhs = filter(lambda rule: len(rule.lhs) >= 2 and len(rule.rhs) == 1, final_rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{num_baths_2, num_beds_3} -> {size_per_room_binned_str_3} (conf: 0.633, supp: 0.187, lift: 1.303, conv: 1.400)
{num_baths_2, size_per_room_binned_str_3} -> {num_beds_3} (conf: 0.691, supp: 0.187, lift: 1.759, conv: 1.963)
{num_beds_2, size_per_room_binned_str_2} -> {num_baths_2} (conf: 0.899, supp: 0.104, lift: 1.791, conv: 4.950)
{num_baths_2, num_beds_2} -> {size_per_room_binned_str_2} (conf: 0.673, supp: 0.104, lift: 2.024, conv: 2.039)
{num_baths_2, num_beds_2} -> {size_per_room_binned_str_2} (conf: 0.673, supp: 0.104, lift: 2.024, conv: 2.039)
{num_baths_2, size_per_room_binned_str_2} -> {num_beds_2} (conf: 0.497, supp: 0.104, lift: 2.277, conv: 1.553)


In [109]:
df_train2

,size_per_room_binned_str,num_baths_str,num_beds_str,size_sqft_str,num_shopping_malls_str
0,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1115,num_shopping_malls_2
1,size_per_room_binned_str_3,num_baths_2,num_beds_4,size_sqft_1575,num_shopping_malls_3
2,size_per_room_binned_str_4,num_baths_6,num_beds_4,size_sqft_3070,num_shopping_malls_5
3,size_per_room_binned_str_2,num_baths_2,num_beds_3,size_sqft_958,num_shopping_malls_4
4,size_per_room_binned_str_3,num_baths_1,num_beds_2,size_sqft_732,num_shopping_malls_16
...,...,...,...,...,...
20091,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_635,num_shopping_malls_4
20092,size_per_room_binned_str_3,num_baths_2,num_beds_2,size_sqft_883,num_shopping_malls_6
20093,size_per_room_binned_str_2,num_baths_4,num_beds_4,size_sqft_1378,num_shopping_malls_5
20094,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1205,num_shopping_malls_8


**Recommend from train data using the rules derived from test data samples**

In [110]:
rules_rhs = filter(lambda rule: len(rule.lhs) >= 2 and len(rule.rhs) == 1, final_rules)


rules_set = []

for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(set(rule.lhs + rule.rhs)) # Prints the rule and its confidence, support, lift, ...
  rules_set.append(set(rule.lhs + rule.rhs))


   
df_train2['flag'] = df_train2.apply(lambda row : 
set((row['size_per_room_binned_str'],
                     row['num_baths_str'],
                      row['size_sqft_str'],
                      row['num_beds_str'],
                       row['num_shopping_malls_str']
                       )) ,axis = 1)





{'num_baths_2', 'num_beds_3', 'size_per_room_binned_str_3'}
{'num_baths_2', 'num_beds_3', 'size_per_room_binned_str_3'}
{'num_baths_2', 'num_beds_2', 'size_per_room_binned_str_2'}
{'num_baths_2', 'num_beds_2', 'size_per_room_binned_str_2'}
{'num_baths_2', 'num_beds_2', 'size_per_room_binned_str_2'}
{'num_baths_2', 'num_beds_2', 'size_per_room_binned_str_2'}


In [111]:
def filterTrain(rules_set, row_set):
   for rule in rules_set:
      # print(rule, row_set)
      if rule.issubset(row_set):
         return True
   return False
top_recommendations = df_train2[df_train2.apply(lambda row: filterTrain(rules_set, row['flag']), axis=1 )]
# top_recommendations.head(2)
# df_train
# top_recommendations
top_recommendations


,size_per_room_binned_str,num_baths_str,num_beds_str,size_sqft_str,num_shopping_malls_str,flag
0,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1115,num_shopping_malls_2,"{num_baths_2, size_per_room_binned_str_3, num_..."
5,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_689,num_shopping_malls_2,"{num_baths_2, num_beds_2, size_per_room_binned..."
6,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1076,num_shopping_malls_6,"{num_baths_2, size_per_room_binned_str_3, num_..."
15,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1119,num_shopping_malls_8,"{num_baths_2, num_shopping_malls_8, size_per_r..."
27,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_721,num_shopping_malls_6,"{num_baths_2, num_beds_2, size_per_room_binned..."
...,...,...,...,...,...,...
20080,size_per_room_binned_str_3,num_baths_2,num_beds_3,size_sqft_1001,num_shopping_malls_4,"{num_baths_2, size_per_room_binned_str_3, size..."
20082,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_720,num_shopping_malls_5,"{num_baths_2, num_beds_2, size_per_room_binned..."
20086,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_797,num_shopping_malls_2,"{num_baths_2, num_beds_2, size_per_room_binned..."
20091,size_per_room_binned_str_2,num_baths_2,num_beds_2,size_sqft_635,num_shopping_malls_4,"{num_baths_2, size_sqft_635, num_beds_2, size_..."


In [112]:
df_train[df_train.index.isin(top_recommendations.index)]

,property_type,built_year,num_beds,num_baths,size_sqft,total_num_units,lat,lng,subzone,planning_area,...,mean_property_sqft,mean_planning_sqft,planning_area_mean,total_rooms,size_per_room,mean_property_type,distance,num_shopping_malls,tenure_99-year leasehold,tenure_freehold
0,hdb,1988,3.0,2.0,1115,116,1.414399,103.837196,yishun south,yishun,...,1079.757868,1231.474343,1.143726e+06,5.0,223.00,6.620452e+05,0.573567,2.0,1,0
5,condo,2024,2.0,2.0,689,633,1.339338,103.763893,bukit batok south,bukit batok,...,1154.798804,1273.031690,1.612262e+06,4.0,172.25,2.919816e+06,1.330448,2.0,0,1
6,condo,2026,3.0,2.0,1076,407,1.310640,103.852149,kampong java,kallang,...,1154.798804,1106.259542,2.256485e+06,5.0,215.20,2.919816e+06,0.400608,6.0,1,0
15,hdb,1992,3.0,2.0,1119,129,1.381630,103.742102,peng siang,choa chu kang,...,1079.757868,1276.588235,7.643136e+05,5.0,223.80,6.620452e+05,0.483884,8.0,1,0
27,hdb,1985,2.0,2.0,721,32,1.355395,103.953221,tampines east,tampines,...,1079.757868,1602.201681,1.392370e+06,4.0,180.25,6.620452e+05,0.180222,6.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080,hdb,2017,3.0,2.0,1001,503,1.400005,103.889859,anchorvale,sengkang,...,1079.757868,1107.429457,1.000985e+06,5.0,200.20,6.620452e+05,1.111874,4.0,1,0
20082,hdb,1978,2.0,2.0,720,375,1.320406,103.884263,aljunied,geylang,...,1079.757868,1186.854945,1.444851e+06,4.0,180.00,6.620452e+05,0.464542,5.0,1,0
20086,hdb,1983,2.0,2.0,797,375,1.424190,103.844932,yishun east,yishun,...,1079.757868,1231.474343,1.143726e+06,4.0,199.25,6.620452e+05,1.248138,2.0,1,0
20091,condo,2026,2.0,2.0,635,605,1.385938,103.834466,tagore,ang mo kio,...,1154.798804,1634.952055,2.355193e+06,4.0,158.75,2.919816e+06,0.150007,4.0,1,0
